In [1]:
!gdown  18cij6foP2ilw2NtadINSnYP6UUbenXCf

!unzip scc5966.zip

import pandas as pd 

df_movie_reviews = pd.read_csv('movie_reviews.csv')

df_movies_data = pd.read_csv('movies_data.csv')

df_test_data = pd.read_csv('test_data.csv')

df_train_data = pd.read_csv('train_data .csv')

df_users_data = pd.read_csv('users_data.csv')

Downloading...
From: https://drive.google.com/uc?id=18cij6foP2ilw2NtadINSnYP6UUbenXCf
To: /content/scc5966.zip
100% 18.4M/18.4M [00:00<00:00, 147MB/s]
Archive:  scc5966.zip
  inflating: movie_reviews.csv       
  inflating: movies_data.csv         
  inflating: test_data.csv           
  inflating: train_data .csv         
  inflating: users_data.csv          


In [5]:
import numpy as np

def similarity_itens(similarities_items, similarities_genres, neighbor_item, item):
  
  if (item,neighbor_item) not in similarities_items.keys():
    similarity = (similarities_items[(neighbor_item,item)] + similarities_genres[(neighbor_item,item)])/2
  else:
    similarity = (similarities_items[(item,neighbor_item)] + similarities_genres[(item,neighbor_item)])/2
    
  return similarity
  
def k_neighbors_nearest(df, similarities_items, similarities_genres, k, user, item):
  k_neighbors_similarity = [-2] * k
  k_neighbors = [-1] * k

  for neighbor_item in df.columns:
    
    ni = neighbor_item
    if item != ni and df.loc[user][ni] is not np.nan:
      
      similarity = similarity_itens(similarities_items, similarities_genres, ni, item)

      for i in range(k):
        if similarity > k_neighbors_similarity[i]:
          aux = k_neighbors_similarity[i]
          k_neighbors_similarity[i] = similarity
          similarity = aux

          aux = k_neighbors[i]
          k_neighbors[i] = ni
          ni = aux

  return k_neighbors

def pred(df, similarities_items, similarities_genres, k, user, item):

  k_neighbors = k_neighbors_nearest(df, similarities_items, similarities_genres, k, user, item)
  
  sum = 0
  sum_similarity = 0

  for neighbor_item in k_neighbors:
    if(neighbor_item != -1): #se nao deu o numero maximo de vizinhos mais proximos
      rating_neighbor_item = df.loc[user][neighbor_item]
    
      similarity = similarity_itens(similarities_items, similarities_genres, neighbor_item, item)

      sum_similarity+= similarity
      
      sum+= similarity * rating_neighbor_item
  
  if sum_similarity ==0 and sum ==0:
    return 0
  elif sum_similarity ==0 and sum !=0:
    return sum
  else:
    return sum/sum_similarity

def pred_generic(parameters):
  return pred(parameters[0], parameters[1], parameters[2], parameters[3], parameters[4], parameters[5])
  
def generate(parameters):
  l= []
  
  for item,row in tqdm(df_test_data.iterrows()):
    user = row['user_id']
    item = row['movie_id']
    if user not in df_train_data['user_id'].unique() and item in df_train_data['movie_id'].unique():
      l.append(items_mean[item])
    elif user in df_train_data['user_id'].unique() and item not in df_train_data['movie_id'].unique():
      l.append(user_mean[user])
    elif user not in df_train_data['user_id'].unique() and item not in df_train_data['movie_id'].unique():
      l.append(global_mean)
    else:
      parameters.append(user)
      parameters.append(item)

      value = pred_generic(parameters)
      if value < 1:
        value = 1
      if value > 5:
        value = 5

      l.append(value)
  
  return l

import pickle

with open('/content/drive/MyDrive/USP/Doctorate/Credits/Recomender_Systems/similarities/total/dic_sim_items_genres_jaccard.p', 'rb') as fp:
    data_genres = pickle.load(fp)

import pickle

with open('/content/drive/MyDrive/USP/Doctorate/Credits/Recomender_Systems/similarities/total/dic_sim_itens_pearson.p', 'rb') as fp:
    data_items = pickle.load(fp)


df_user_item = pd.DataFrame(index = df_train_data['user_id'].unique(), columns=df_train_data['movie_id'].unique())

for movie in df_train_data['movie_id'].unique():
  temp = df_train_data[df_train_data.movie_id == movie]
  df_user_item.loc[temp.user_id, movie] = temp.rating.to_list()

items_mean = np.mean(df_user_item)
user_mean = np.mean(df_user_item.T)
global_mean = np.nanmean(df_user_item.values.tolist())

In [ ]:
from tqdm import tqdm
k = 3
  
y_pred = generate([df_user_item, data_items, data_genres, k])
  
df_test_data['rating'] = y_pred
df_rating = df_test_data[['id','rating']]
df_rating = df_rating.set_index('id')
name = 'FH_' + str(k) + '.csv'
df_rating.to_csv('/content/drive/MyDrive/USP/Doctorate/Credits/Recomender_Systems/results-kaggle/' +name)